In [4]:
#Import neccessary libraries
import numpy as np
import pandas as pd
import openai
import time
import random
from sklearn.metrics.pairwise import cosine_similarity
# Initialize the OpenAI library using your API key
openai.api_key = "your_openai_api_key"

In [2]:
# to get the pre-processed main product data file
df = pd.read_csv('data.csv')
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,final
0,0,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,productDisplayName for gender's usage wear in ...
1,1,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,Turtle Check Men Navy Blue Shirt for Men's Cas...
2,2,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,Peter England Men Party Blue Jeans for Men's C...
3,3,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,Titan Women Silver Watch for Women's Casual we...
4,4,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,Manchester United Men Solid Black Track Pants ...
...,...,...,...,...,...,...,...,...,...,...,...,...
44442,44442,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013,Casual,Gas Men Caddy Casual Shoe,Gas Men Caddy Casual Shoe for Men's Casual wea...
44443,44443,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011,Casual,Lotto Men's Soccer Track Flip Flop,Lotto Men's Soccer Track Flip Flop for Men's C...
44444,44444,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011,Casual,Puma Men Graphic Stellar Blue Tshirt,Puma Men Graphic Stellar Blue Tshirt for Men's...
44445,44445,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017,Casual,Rasasi Women Blue Lady Perfume,Rasasi Women Blue Lady Perfume for Women's Cas...


In [3]:
#cleaning the data by removing unwanted columns and rows
df.columns = df.iloc[0]
df = df[1:]
del df[df.columns[0]]
df


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,productDisplayName for gender's usage wear in year for season season
1,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,Turtle Check Men Navy Blue Shirt for Men's Cas...
2,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,Peter England Men Party Blue Jeans for Men's C...
3,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,Titan Women Silver Watch for Women's Casual we...
4,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,Manchester United Men Solid Black Track Pants ...
5,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,Puma Men Grey T-shirt for Men's Casual wear in...
...,...,...,...,...,...,...,...,...,...,...,...
44442,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013,Casual,Gas Men Caddy Casual Shoe,Gas Men Caddy Casual Shoe for Men's Casual wea...
44443,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011,Casual,Lotto Men's Soccer Track Flip Flop,Lotto Men's Soccer Track Flip Flop for Men's C...
44444,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011,Casual,Puma Men Graphic Stellar Blue Tshirt,Puma Men Graphic Stellar Blue Tshirt for Men's...
44445,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017,Casual,Rasasi Women Blue Lady Perfume,Rasasi Women Blue Lady Perfume for Women's Cas...


In [4]:
#renaming the columns of the dataframe for ease of usage in future cases
df.rename(columns = {'productDisplayName for gender\'s usage wear in year for season season':'product'}, inplace = True)

C:\Users\lucif\AppData\Local\Temp\ipykernel_3688\3813040040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'productDisplayName for gender\'s usage wear in year for season season':'product'}, inplace = True)


In [6]:
#creating a sample dataframe using a subset of the original dataset
sub_df = df.head(100)
sub_df.to_csv('sample.csv', index=False)
sub_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,product
1,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,Turtle Check Men Navy Blue Shirt for Men's Cas...
2,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,Peter England Men Party Blue Jeans for Men's C...
3,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,Titan Women Silver Watch for Women's Casual we...
4,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,Manchester United Men Solid Black Track Pants ...
5,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,Puma Men Grey T-shirt for Men's Casual wear in...
...,...,...,...,...,...,...,...,...,...,...,...
96,25349,Men,Apparel,Innerwear,Briefs,Red,Summer,2016,Casual,Levis Men Red Brief,Levis Men Red Brief for Men's Casual wear in 2...
97,11940,Men,Footwear,Sandal,Sandals,Black,Summer,2011,Casual,Ganuchi Men Casual Black Sandals,Ganuchi Men Casual Black Sandals for Men's Cas...
98,55802,Women,Apparel,Loungewear and Nightwear,Night suits,White,Summer,2017,Casual,SDL by Sweet Dreams Women Green Printed Night ...,SDL by Sweet Dreams Women Green Printed Night ...
99,29584,Men,Apparel,Topwear,Tshirts,Black,Summer,2012,Casual,Indigo Nation Men Printed Black T-shirt,Indigo Nation Men Printed Black T-shirt for Me...


In [5]:
#creating the function for the openai api
def get_embedding(text, model="text-embedding-ada-002"):
    #text = text.replace("\n", " ")
    return openai.Embedding.create(input=[text], model=model)['data'][0]['embedding']

In [14]:
#getting the embeddings for the sample data frame using the get embeddings function for the product column
#WARNING: this will take a while to run, it will take 34-35 mins to run
embeddings = []
for text in sub_df['product']:
    embedding = get_embedding(text, model='text-embedding-ada-002')
    embf = np.array(embedding)
    embeddings.append(embf)
    time.sleep(20)

In [15]:
#adding the embeddings column to the dataframe
sub_df['embedding'] = embeddings
sub_df

C:\Users\lucif\AppData\Local\Temp\ipykernel_3688\2996831028.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['embedding'] = embeddings


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,product,embedding
1,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,Turtle Check Men Navy Blue Shirt for Men's Cas...,"[-0.0076116761192679405, -0.024072913452982903..."
2,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,Peter England Men Party Blue Jeans for Men's C...,"[0.0021638981997966766, -0.02104014717042446, ..."
3,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,Titan Women Silver Watch for Women's Casual we...,"[-0.012933733873069286, -0.020804161205887794,..."
4,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,Manchester United Men Solid Black Track Pants ...,"[-0.0010367854265496135, -0.016627689823508263..."
5,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,Puma Men Grey T-shirt for Men's Casual wear in...,"[0.011330138891935349, -0.006096387282013893, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
96,25349,Men,Apparel,Innerwear,Briefs,Red,Summer,2016,Casual,Levis Men Red Brief,Levis Men Red Brief for Men's Casual wear in 2...,"[0.013804197311401367, -0.006152432411909103, ..."
97,11940,Men,Footwear,Sandal,Sandals,Black,Summer,2011,Casual,Ganuchi Men Casual Black Sandals,Ganuchi Men Casual Black Sandals for Men's Cas...,"[-0.008123484440147877, -0.0007026781095191836..."
98,55802,Women,Apparel,Loungewear and Nightwear,Night suits,White,Summer,2017,Casual,SDL by Sweet Dreams Women Green Printed Night ...,SDL by Sweet Dreams Women Green Printed Night ...,"[0.002484317868947983, -0.021454863250255585, ..."
99,29584,Men,Apparel,Topwear,Tshirts,Black,Summer,2012,Casual,Indigo Nation Men Printed Black T-shirt,Indigo Nation Men Printed Black T-shirt for Me...,"[-0.010722247883677483, -0.01884516328573227, ..."


In [16]:
#saving the data to a csv file for ease of access
sub_df.to_csv('em_sample.csv')

In [19]:
#creating an example past purchase database of a women user by selecting 20 rows at random because we don't have access to such data
customer0 = pd.read_csv('sample.csv')
random_rows = customer0[customer0['gender'] == 'Women'].sample(n=20, random_state=random.seed())
random_rows

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,product
87,56489,Women,Personal Care,Nails,Nail Polish,Bronze,Spring,2017,NaN,Streetwear Ash Nail Polish # 31,Streetwear Ash Nail Polish # 31 for Women's na...
63,42841,Women,Accessories,Bags,Handbags,Beige,Summer,2012,Casual,French Connection Women Beige Handbag,French Connection Women Beige Handbag for Wome...
19,47359,Women,Accessories,Bags,Handbags,Brown,Summer,2012,Casual,Baggit Women Brown Handbag,Baggit Women Brown Handbag for Women's Casual ...
13,47957,Women,Accessories,Bags,Handbags,Blue,Summer,2012,Casual,Murcia Women Blue Handbag,Murcia Women Blue Handbag for Women's Casual w...
18,51832,Women,Apparel,Innerwear,Bra,Beige,Summer,2016,Casual,Bwitch Beige Full-Coverage Bra BW335,Bwitch Beige Full-Coverage Bra BW335 for Women...
41,2886,Women,Footwear,Shoes,Flats,Brown,Winter,2015,Casual,Catwalk Women Leather Brown Flats,Catwalk Women Leather Brown Flats for Women's ...
36,58183,Women,Accessories,Bags,Handbags,White,Summer,2012,Casual,Rocky S Women White Handbag,Rocky S Women White Handbag for Women's Casual...
94,39716,Women,Apparel,Dress,Dresses,Blue,Summer,2012,Casual,Arrow Woman Women Blue Dress,Arrow Woman Women Blue Dress for Women's Casua...
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,Titan Women Silver Watch for Women's Casual we...
97,55802,Women,Apparel,Loungewear and Nightwear,Night suits,White,Summer,2017,Casual,SDL by Sweet Dreams Women Green Printed Night ...,SDL by Sweet Dreams Women Green Printed Night ...


In [21]:
#genrating embeddings from the customer data and appending it as a new column in the customer data
#WARNING : It takes a lot of time to run this snippet. It may take upto 7 minutes.
customer_embeddings = []
for text in customer_data['product']:
    embedding = get_embedding(text, model='text-embedding-ada-002')
    customer0emb = np.array(embedding)
    customer_embeddings.append(customer0emb)
    time.sleep(20)  

customer_data['embedding'] = customer_embeddings

In [23]:
#saving as csv for ease of access
customer_data.to_csv("customer0_data.csv", index=False)

In [ ]:
customer_df = pd.read_csv("customer0_data.csv")
product_df = pd.read_csv("em_sample.csv")

In [8]:
customer_df


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,product,embedding
0,56489,Women,Personal Care,Nails,Nail Polish,Bronze,Spring,2017,NaN,Streetwear Ash Nail Polish # 31,Streetwear Ash Nail Polish # 31 for Women's na...,[ 0.00020446 -0.00243809 -0.00203039 ... -0.00...
1,42841,Women,Accessories,Bags,Handbags,Beige,Summer,2012,Casual,French Connection Women Beige Handbag,French Connection Women Beige Handbag for Wome...,[-0.01458547 0.01979456 -0.01336133 ... -0.00...
2,47359,Women,Accessories,Bags,Handbags,Brown,Summer,2012,Casual,Baggit Women Brown Handbag,Baggit Women Brown Handbag for Women's Casual ...,[-0.02690023 -0.0028223 -0.00848654 ... -0.02...
3,47957,Women,Accessories,Bags,Handbags,Blue,Summer,2012,Casual,Murcia Women Blue Handbag,Murcia Women Blue Handbag for Women's Casual w...,[-0.01254059 0.00562702 -0.00445743 ... -0.01...
4,51832,Women,Apparel,Innerwear,Bra,Beige,Summer,2016,Casual,Bwitch Beige Full-Coverage Bra BW335,Bwitch Beige Full-Coverage Bra BW335 for Women...,[-0.01525669 0.0008875 0.00405854 ... -0.02...
5,2886,Women,Footwear,Shoes,Flats,Brown,Winter,2015,Casual,Catwalk Women Leather Brown Flats,Catwalk Women Leather Brown Flats for Women's ...,[-0.01321578 0.00508073 -0.01083368 ... -0.01...
6,58183,Women,Accessories,Bags,Handbags,White,Summer,2012,Casual,Rocky S Women White Handbag,Rocky S Women White Handbag for Women's Casual...,[-0.01063765 -0.00130036 -0.01578364 ... -0.01...
7,39716,Women,Apparel,Dress,Dresses,Blue,Summer,2012,Casual,Arrow Woman Women Blue Dress,Arrow Woman Women Blue Dress for Women's Casua...,[-0.02652567 -0.00798232 -0.02294917 ... -0.00...
8,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,Titan Women Silver Watch for Women's Casual we...,[-0.01293373 -0.02080416 -0.01633113 ... -0.00...
9,55802,Women,Apparel,Loungewear and Nightwear,Night suits,White,Summer,2017,Casual,SDL by Sweet Dreams Women Green Printed Night ...,SDL by Sweet Dreams Women Green Printed Night ...,[ 0.00248432 -0.02145486 0.01778531 ... -0.00...


In [11]:
product_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,product,embedding
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,Turtle Check Men Navy Blue Shirt for Men's Cas...,[-0.00761168 -0.02407291 -0.00664046 ... -0.00...
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,Peter England Men Party Blue Jeans for Men's C...,[ 0.0021639 -0.02104015 -0.00901813 ... 0.00...
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,Titan Women Silver Watch for Women's Casual we...,[-0.01293373 -0.02080416 -0.01633113 ... -0.00...
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,Manchester United Men Solid Black Track Pants ...,[-0.00103679 -0.01662769 -0.00521979 ... -0.00...
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,Puma Men Grey T-shirt for Men's Casual wear in...,[ 0.01133014 -0.00609639 -0.00284219 ... -0.00...
...,...,...,...,...,...,...,...,...,...,...,...,...
95,25349,Men,Apparel,Innerwear,Briefs,Red,Summer,2016,Casual,Levis Men Red Brief,Levis Men Red Brief for Men's Casual wear in 2...,[ 0.0138042 -0.00615243 0.00089831 ... -0.01...
96,11940,Men,Footwear,Sandal,Sandals,Black,Summer,2011,Casual,Ganuchi Men Casual Black Sandals,Ganuchi Men Casual Black Sandals for Men's Cas...,[-0.00812348 -0.00070268 0.00615664 ... -0.04...
97,55802,Women,Apparel,Loungewear and Nightwear,Night suits,White,Summer,2017,Casual,SDL by Sweet Dreams Women Green Printed Night ...,SDL by Sweet Dreams Women Green Printed Night ...,[ 0.00248432 -0.02145486 0.01778531 ... -0.00...
98,29584,Men,Apparel,Topwear,Tshirts,Black,Summer,2012,Casual,Indigo Nation Men Printed Black T-shirt,Indigo Nation Men Printed Black T-shirt for Me...,[-0.01072225 -0.01884516 -0.00674527 ... -0.00...


In [12]:
#User Input Query
query = input("User: ")

In [13]:
#getting embeddings from the query entered by the customer
response = openai.Embedding.create(
    input=query,
    model="text-embedding-ada-002"
)
embeddings_customer_query = response['data'][0]['embedding']

embeddings_customer_query = np.array(embeddings_customer_query)


In [15]:
# #finding the cosine similarity between customer query and purchase history and adding it to the data frame
# #sorting the data based on the cosine similarity(High to Low)

embeddings_customer_query = embeddings_customer_query.reshape(1, -1)
customer_df['search_purchase_history'] = customer_df['embedding'].apply(lambda x: cosine_similarity([x], [embeddings_customer_query])[0][0])
customer_df = customer_df.sort_values('search_purchase_history', ascending=False)
customer_df

In [40]:
#finding the cosine similarity between customer query and product list and adding it to the data frame
#sorting the data based on the cosine similarity(High to Low)
product_df['search_products'] = product_df['embedding'].apply(lambda x: cosine_similarity([x], [embeddings_customer_query])[0][0])
product_df = product_df.sort_values('search_products', ascending=False)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,product,embedding,search_purchase_history,search_products
62,15528,Men,Apparel,Topwear,Tshirts,Black,Fall,2011,Casual,Puma Men Scribble Black Tshirts,Puma Men Scribble Black Tshirts for Men's Casu...,"[-0.0029755951836705208, -0.01764657348394394,...",0.826121,0.826121
99,29584,Men,Apparel,Topwear,Tshirts,Black,Summer,2012,Casual,Indigo Nation Men Printed Black T-shirt,Indigo Nation Men Printed Black T-shirt for Me...,"[-0.010722247883677483, -0.01884516328573227, ...",0.825537,0.825537
56,5891,Men,Apparel,Topwear,Tshirts,Black,Summer,2011,Casual,Puma Men's Stripe Polo Black T-shirt,Puma Men's Stripe Polo Black T-shirt for Men's...,"[-0.0010690178023651242, -0.006576103623956442...",0.825239,0.825239
74,2288,Women,Apparel,Topwear,Tshirts,Black,Fall,2010,Sports,Nike Women Black T-shirt,Nike Women Black T-shirt for Women's Sports we...,"[-0.022338490933179855, -0.03431522846221924, ...",0.816452,0.816452
16,12369,Men,Apparel,Topwear,Shirts,Purple,Fall,2011,Formal,Reid & Taylor Men Check Purple Shirts,Reid & Taylor Men Check Purple Shirts for Men'...,"[0.0027601502370089293, -0.029423994943499565,...",0.813133,0.813133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,Titan Women Silver Watch for Women's Casual we...,"[-0.012933733873069286, -0.020804161205887794,...",0.745923,0.745923
86,48781,Women,Accessories,Jewellery,Pendant,Silver,Summer,2012,Casual,Lucera Women Silver Pendant,Lucera Women Silver Pendant for Women's Casual...,"[-0.010162770748138428, 0.0168339554220438, -0...",0.745623,0.745623
39,51658,Women,Accessories,Watches,Watches,Copper,Winter,2016,Casual,Fossil Women Copper Chronograph Watch CH2589,Fossil Women Copper Chronograph Watch CH2589 f...,"[-0.01265791431069374, 0.004657279700040817, 0...",0.744140,0.744140
100,43967,Women,Personal Care,Fragrance,Perfume and Body Mist,Red,Spring,2017,Casual,DKNY Women Red Delicious Perfume,DKNY Women Red Delicious Perfume for Women's C...,"[-0.005608072504401207, 0.0035180270206183195,...",0.742782,0.742782


In [42]:
product_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,product,embedding,search_products
62,15528,Men,Apparel,Topwear,Tshirts,Black,Fall,2011,Casual,Puma Men Scribble Black Tshirts,Puma Men Scribble Black Tshirts for Men's Casu...,"[-0.0029755951836705208, -0.01764657348394394,...",0.826121
99,29584,Men,Apparel,Topwear,Tshirts,Black,Summer,2012,Casual,Indigo Nation Men Printed Black T-shirt,Indigo Nation Men Printed Black T-shirt for Me...,"[-0.010722247883677483, -0.01884516328573227, ...",0.825537
56,5891,Men,Apparel,Topwear,Tshirts,Black,Summer,2011,Casual,Puma Men's Stripe Polo Black T-shirt,Puma Men's Stripe Polo Black T-shirt for Men's...,"[-0.0010690178023651242, -0.006576103623956442...",0.825239
74,2288,Women,Apparel,Topwear,Tshirts,Black,Fall,2010,Sports,Nike Women Black T-shirt,Nike Women Black T-shirt for Women's Sports we...,"[-0.022338490933179855, -0.03431522846221924, ...",0.816452
16,12369,Men,Apparel,Topwear,Shirts,Purple,Fall,2011,Formal,Reid & Taylor Men Check Purple Shirts,Reid & Taylor Men Check Purple Shirts for Men'...,"[0.0027601502370089293, -0.029423994943499565,...",0.813133
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,Titan Women Silver Watch for Women's Casual we...,"[-0.012933733873069286, -0.020804161205887794,...",0.745923
86,48781,Women,Accessories,Jewellery,Pendant,Silver,Summer,2012,Casual,Lucera Women Silver Pendant,Lucera Women Silver Pendant for Women's Casual...,"[-0.010162770748138428, 0.0168339554220438, -0...",0.745623
39,51658,Women,Accessories,Watches,Watches,Copper,Winter,2016,Casual,Fossil Women Copper Chronograph Watch CH2589,Fossil Women Copper Chronograph Watch CH2589 f...,"[-0.01265791431069374, 0.004657279700040817, 0...",0.744140
100,43967,Women,Personal Care,Fragrance,Perfume and Body Mist,Red,Spring,2017,Casual,DKNY Women Red Delicious Perfume,DKNY Women Red Delicious Perfume for Women's C...,"[-0.005608072504401207, 0.0035180270206183195,...",0.742782


In [43]:
#getting the top 3 recommendation from the sorted product data frame
top_3_products_df = product_df.head(3)
top_3_products_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,product,embedding,search_products
62,15528,Men,Apparel,Topwear,Tshirts,Black,Fall,2011,Casual,Puma Men Scribble Black Tshirts,Puma Men Scribble Black Tshirts for Men's Casu...,"[-0.0029755951836705208, -0.01764657348394394,...",0.826121
99,29584,Men,Apparel,Topwear,Tshirts,Black,Summer,2012,Casual,Indigo Nation Men Printed Black T-shirt,Indigo Nation Men Printed Black T-shirt for Me...,"[-0.010722247883677483, -0.01884516328573227, ...",0.825537
56,5891,Men,Apparel,Topwear,Tshirts,Black,Summer,2011,Casual,Puma Men's Stripe Polo Black T-shirt,Puma Men's Stripe Polo Black T-shirt for Men's...,"[-0.0010690178023651242, -0.006576103623956442...",0.825239


In [44]:
#getting the top 3 recommendation from the sorted customer's past purchases data frame
top_3_purchases_df = customer_df.head(3)
top_3_purchases_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,product,embedding,search_purchase_history
13,2288,Women,Apparel,Topwear,Tshirts,Black,Fall,2010,Sports,Nike Women Black T-shirt,Nike Women Black T-shirt for Women's Sports we...,"[-0.022314177826046944, -0.03431757539510727, ...",0.816341
11,26960,Women,Apparel,Topwear,Shirts,Purple,Summer,2012,Casual,Jealous 21 Women Purple Shirt,Jealous 21 Women Purple Shirt for Women's Casu...,"[-0.02039269171655178, -0.014512669295072556, ...",0.800149
15,38402,Women,Accessories,Belts,Belts,Black,Summer,2012,Casual,Baggit Women Hood Dabang Black Belt,Baggit Women Hood Dabang Black Belt for Women'...,"[-0.029186759144067764, -0.005480786319822073,...",0.793769


In [52]:
#creating the chat bot
message_objects = []
message_objects.append({"role":"system", "content":"You're a chatbot helping customers with shopping-related questions and helping them with product recommendations"})

In [53]:
message_objects.append({"role":"user", "content": query})

In [54]:
prev_purchases = ". ".join([f"{row['product']}" for index, row in top_3_purchases_df.iterrows()])
prev_purchases

"Nike Women Black T-shirt for Women's Sports wear in 2010 for Fall season. Jealous 21 Women Purple Shirt for Women's Casual wear in 2012 for Summer season. Baggit Women Hood Dabang Black Belt for Women's Casual wear in 2012 for Summer season"

In [55]:
message_objects.append({"role":"user", "content": f"Here're my latest product orders: {prev_purchases}"})

In [56]:
message_objects.append({"role":"user", "content": f"Please give me a detailed explanation of your recommendations"})
message_objects.append({"role":"user", "content": "Please be friendly and talk to me like a person, don't just give me a list of recommendations"})

In [57]:
message_objects.append({"role": "assistant", "content": f"I found these 3 products I would recommend"})

In [59]:
products_list = []

for index, row in top_3_products_df.iterrows():
    brand_dict = {'role': "assistant", "content": f"{row['product']}"}
    products_list.append(brand_dict)


[{'role': 'assistant',
  'content': "Puma Men Scribble Black Tshirts for Men's Casual wear in 2011 for Fall season"},
 {'role': 'assistant',
  'content': "Indigo Nation Men Printed Black T-shirt for Men's Casual wear in 2012 for Summer season"},
 {'role': 'assistant',
  'content': "Puma Men's Stripe Polo Black T-shirt for Men's Casual wear in 2011 for Summer season"}]

In [60]:
message_objects.extend(products_list)

In [61]:
message_objects.append({"role": "assistant", "content":"Here's my summarized recommendation of products, and why it would suit you:"})

In [64]:
#genrating the final output in a readable format 
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=message_objects
)

print(completion.choices[0].message['content'])

1) Nike Women's Black T-shirt for Women's Sports wear in 2010 (Fall season): This is a great option for sports activities as it is designed for comfort and mobility. Nike is a trusted brand known for their quality and durability in sportswear.

2) Jealous 21 Women's Purple Shirt for Women's Casual wear in 2012 (Summer season): While not in black, this purple shirt can add a pop of color to your casual wardrobe. Jealous 21 is a popular brand known for trendy and stylish clothing, so this shirt is perfect for casual outings during the summer.

3) Baggit Women's Hood Dabang Black Belt for Women's Casual wear in 2012 (Summer season): This black belt from Baggit is a great accessory to complete your casual outfit. It adds a touch of style and can be paired with jeans or dresses. Baggit is a trusted brand for accessories and their products are known for their quality and durability.

I hope this helps you in making your decision. Let me know if there's anything else I can assist you with!
